import torch

In [34]:
import torch

reference: `https://huggingface.co/docs/datasets/process`

In [12]:
from datasets import load_dataset

ds = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['highlights'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['highlights'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['highlights'],
        num_rows: 11490
    })
})

In [15]:
# remove columns
ds = ds.remove_columns(["article", "id"])
ds

DatasetDict({
    train: Dataset({
        features: ['highlights'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['highlights'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['highlights'],
        num_rows: 11490
    })
})

In [17]:
# rename columns
ds = ds.rename_column("highlights", "input")

In [ ]:
# mapping 
from datasets import Dataset

# Function to split the string without breaking words
def split_example(example):
    text = example["input"]
    words = text.split()
    mid = len(words) // 2  # Find the midpoint by words
    return {"input": " ".join(words[:mid]), "output_example": " ".join(words[mid:])}

updated_ds = ds.map(split_example, with_indices=True)
updated_ds




In [32]:
# mapping (+ how to set id column)
from datasets import Dataset



# Function to split the string without breaking words
def split_example(example, idx):
    text = example["input"]
    words = text.split()
    mid = len(words) // 2  # Find the midpoint by words
    return {"id": idx,"input": " ".join(words[:mid]), "output_example": " ".join(words[mid:])}

updated_ds = ds.map(split_example, with_indices=True, num_proc=100)
updated_ds




Map (num_proc=100):   0%|          | 0/287113 [00:00<?, ? examples/s]

Map (num_proc=100):   0%|          | 0/13368 [00:00<?, ? examples/s]

Map (num_proc=100):   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 11490
    })
})

In [43]:
updated_ds

DatasetDict({
    train: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['input', 'id', 'output_example'],
        num_rows: 11490
    })
})

In [47]:

from datasets import load_dataset, DatasetDict


# 데이터셋의 각 부분에서 원하는 퍼센트만 선택
percentage = 0.0001  # 예: 전체 데이터에서 30%만 남기기

def sample_dataset(dataset, percentage):
    sampled = dataset.shuffle(seed=42).select(range(int(len(dataset) * percentage)))
    return sampled

# train, test, validation 각각에 대해 적용
train_dataset = sample_dataset(updated_ds['train'], percentage)
test_dataset = sample_dataset(updated_ds['test'], percentage)
val_dataset = sample_dataset(updated_ds['validation'], percentage)

# 새로운 DatasetDict로 결합
new_dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': val_dataset
})

# 결과 확인
print(f"New Train dataset size: {len(new_dataset_dict['train'])}")
print(f"New Test dataset size: {len(new_dataset_dict['test'])}")
print(f"New Validation dataset size: {len(new_dataset_dict['validation'])}")

New Train dataset size: 28
New Test dataset size: 1
New Validation dataset size: 1


In [48]:
# 저장
new_dataset_dict.save_to_disk('./datasets/example_LM_input')

Saving the dataset (0/1 shards):   0%|          | 0/28 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
import torch
from multiprocess import set_start_method
from transformers import AutoTokenizer, AutoModelForCausalLM 
from datasets import load_dataset

# Get an example dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts", split="train")

# Get an example model and its tokenizer 
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat").eval()
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")
def gpu_computation(batch, rank):
    # Move the model on the right GPU if it's not there already
    device = f"cuda:{(rank or 0) % torch.cuda.device_count()}"
    model.to(device)
    
    # Your big GPU call goes here, for example:
    chats = [[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ] for prompt in batch["prompt"]]
    texts = [tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True
    ) for chat in chats]
    model_inputs = tokenizer(texts, padding=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**model_inputs, max_new_tokens=512)
    batch["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch
if __name__ == "__main__":
    set_start_method("spawn")
    updated_dataset = dataset.map(
        gpu_computation,
        batched=True,
        batch_size=16,
        with_rank=True,
        num_proc=torch.cuda.device_count(),  # one process per GPU
    )

In [36]:
torch.cuda.device_count()

0

In [ ]:
import torch
import os
from multiprocess import set_start_method
from transformers import AutoTokenizer, AutoModelForCausalLM 
from datasets import load_dataset

# Get an example dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts", split="train")

# Get an example model and its tokenizer
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat").eval()
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")

def cpu_computation(batch, rank=None):
    # Use CPU (no GPU involved here)
    device = torch.device("cpu")
    model.to(device)
    
    # Your computation goes here
    chats = [[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ] for prompt in batch["prompt"]]
    
    texts = [tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True
    ) for chat in chats]
    
    model_inputs = tokenizer(texts, padding=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(**model_inputs, max_new_tokens=512)
    
    batch["output"] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

if __name__ == "__main__":
    set_start_method("spawn")
    
    # Get the number of CPU cores
    num_cpu_cores = os.cpu_count()

    updated_dataset = dataset.map(
        cpu_computation,
        batched=True,
        batch_size=16,
        num_proc=num_cpu_cores,  # Use all CPU cores
    )

In [40]:
import os
os.cpu_count()

16